# OpenCV fundamentals

This notebook covers opening files, looking at pixels, and some simple image processing techniques.

We'll use the following sample image, stolen from the Internet. But you can use whatever image you like.

![Brain MRI](https://raw.githubusercontent.com/chmdznr/ml_image_potekkes/main/data/Brain_MRI_0077_13.jpg "Sample brain MRI")

## Python getting started

First we need to import the relevant libraries: OpenCV itself, Numpy, and a couple of others. Common and Video are simple data handling and opening routines that you can find in the OpenCV Python Samples directory or other sources.  We'll start each notebook with the same includes - you don't need all of them every time (so this is bad form, really) but it's easier to just copy and paste. 